In [14]:
import pandas as pd
#import hockey_rink as hr
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math



In [20]:
# read in the data
df = pd.read_csv('../data/events.csv')


In [16]:
# each row has a column EventStartCoordinate and EventEndCoordinate which if applicable, contains the x and y coordinates of the event.
# these columns need to be split into two columns, one for x and one for y
df[['EventStartX', 'EventStartY']] = df['EventStartCoordinate'].str.split(',', expand=True)
df[['EventEndX', 'EventEndY']] = df['EventEndCoordinate'].str.split(',', expand=True)
# convert the columns to numeric
df['EventStartX'] = pd.to_numeric(df['EventStartX'])
df['EventStartY'] = pd.to_numeric(df['EventStartY'])
df['EventEndX'] = pd.to_numeric(df['EventEndX'])
df['EventEndY'] = pd.to_numeric(df['EventEndY'])

df

In [18]:
# Functions

# Euclidean distance between two points
def distance(point1, point2):
    return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

In [19]:
# just taking the event with type shot or pass
dfshot_goal = df[(df['EventType'] == 'Shot') | (df['EventType'] == 'Pass')]

for index, event in dfshot_goal.head().iterrows():
    
    #print(f"Index: {index}")
    #print(f"Name: {event['EventType']}")
    
    
    

    # Function to calculate the Euclidean distance between two points
    def distance(point1, point2):
        return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)
    
    playerNumbers = []

    primary_player_id = event['EventPrimaryPlayerId']
    
    # skip nan values
    if len(df.columns[event == primary_player_id].drop('EventPrimaryPlayerId')) == 0:
        continue
    else:
        primary_player_number = df.columns[event == primary_player_id].drop('EventPrimaryPlayerId')[0][13]
    
    goal_coords = [85,]
    
    if event['EventPrimaryTeam'] == "Home":
        primary_player_team = "Home"
        opponent_team = "Away"
    elif event['EventPrimaryTeam'] == "Away":
        primary_player_team = "Away"
        opponent_team = "Home"
    else:
        print("Error: The team is neither home nor away.")
    
    distances_to_opponents = []
    for i in range(1, 12):

        coords_primary = event[f'StartPlayerCoordinates{primary_player_number}'].split(',')
        float_coords_primary = [float(coord) for coord in coords_primary]
        
        #print(event[f'StartPlayerTeam{i}'])
        if event[f'StartPlayerTeam{i}'] == opponent_team: 
            
            primary_player_id = event['EventPrimaryPlayerId']
            
            #print(df.columns[event == primary_player_id])
           
            coords_player_cur = event[f'StartPlayerCoordinates{i}'].split(',')
            float_coords_player_cur = [float(coord) for coord in coords_player_cur]
            distances_to_opponents.append(distance(float_coords_primary, float_coords_player_cur))

    mean_distance = sum(distances_to_opponents)/len(distances_to_opponents)
    
    print("This is the mean distance of all opponents players to the primary player: " + str(mean_distance))
    
    
    distance_to_goal = 0.0
    
    # determine which goal is correct one

    print(event)
    if float(event['EventStartX']) > 0:
        distance_to_goal = distance([26.0,0.0], float_coords_player_cur)
    else:
        distance_to_goal = distance([-26.0,0.0], float_coords_player_cur)
    
    
    print("This is the distance to the goal: "+ str(distance_to_goal))
    
    
    
    
    
    
    
        
        
     
           

            
#nickel
   
    
    
    # 
    # if(row['EventType'] == 'Shot'):
    #     test = row['StartPlayerCoordinates4']
    #     print(test)
    
    


    

This is the mean distance of all opponents players to the primary player: 13.141888235827254
Period                                1
MatchClock                            1
Timestamp                 1708857105420
EventType                          Pass
TeamStrengthType           FullStrength
                              ...      
StartPlayerDirection12      -0.18,0.984
EventStartX                        -6.0
EventStartY                         4.7
EventEndX                         -11.4
EventEndY                          -4.1
Name: 3, Length: 118, dtype: object
This is the distance to the goal: 24.689471440271863
This is the mean distance of all opponents players to the primary player: 11.020423200385382
Period                                1
MatchClock                            6
Timestamp                 1708857110740
EventType                          Pass
TeamStrengthType           FullStrength
                              ...      
StartPlayerDirection12     -0.456,-0.89
Event